在使用线性回归时，我们经常需要尝试不同的自变量的组合以更加准确的拟合数据，这既包括尝试不同的自变量、使用交叉项及二次项，也包括对分类变量进行处理等。R语言提供了一套记法，可以让我们能够非常方便的表达我们想要拟合的模型，如使用Y~X就可以表示因变量为Y，自变量为X的带常数项的线性模型，而使用Y ~ X1 + X2 + X1:X2就可以表示因变量为Y，自变量为X1与X2及其交叉项的模型，详细的介绍可以看这篇文章。与之类似，python中statsmodels模块也可以实现同样的功能，从而可以让我们可以在python中更加方便的进行线性回归。首先，我们需要导入一些必需的模块：

In [8]:
import statsmodels.formula.api as sm
import numpy as np
import pandas as pd

我们载入一个非常著名的数据集，这个数据集是R语言自带的，其中包含了1973-1974年出产的32辆不同品牌及型号汽车的油耗与性能方面的数据，具体的信息及每个变量的含义可以参见[这个页面](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/mtcars.html)，也可以[这里](https://gist.github.com/seankross/a412dfbd88b3db70b74b)下载这个数据。下面，我们直接导入这个数据并查看下数据的基本信息：

In [5]:
cars = pd.read_csv('data/mtcars.csv')
cars.head(10)

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


## 一、初步示例

我们来看一个简单的示例，比如我关心数据集中汽车油耗与其重量的关系，常识上我们认为其它情况不变，汽车的重量越重，油耗应该越高，我们可以使用线性回归来验证一下这个猜测。我们使用`mpg`即每一加仑油行驶的英里数为因变量，以`wt`即汽车重量为自变量来做回归：

In [7]:
model = smf.ols(formula='mpg ~ wt',data=cars)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     91.38
Date:                Thu, 04 Oct 2018   Prob (F-statistic):           1.29e-10
Time:                        22:12:36   Log-Likelihood:                -80.015
No. Observations:                  32   AIC:                             164.0
Df Residuals:                      30   BIC:                             167.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     37.2851      1.878     19.858      0.0

从以上回归过程我们可以发现三点：

- statsmodels中回归可以分为三步，分别为定义模型、拟合求解模型及使用`summary()`方法查看回归结果；
- 使用与R语言中类似的方法，我们可以方便的定义我们想要的回归模型；
- wt变量的符号为负，说明汽车重量越重，其每加仑行驶英里数越短，这也我们的猜测一致。从p值来看，wt变量及常数项都是显著的。

为了避免重复写以上代码，我们定义一个函数，将回归的三步曲放到同一个函数中去执行：

In [10]:
def linear_reg(df,formula):
    model = sm.ols(formula=formula, data=df)
    result = model.fit()
    print(result.summary())

## 二、各种操作符的含义